In [1]:
import pickle
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import copy

In [2]:
teams = ['BKN', 'MIL', 'GSW', 'LAL', 'IND', 'CHA', 'CHI', 'DET',
          'WAS', 'TOR', 'BOS', 'NYK', 'CLE', 'MEM', 'PHI', 'NOP',
          'HOU','MIN', 'ORL', 'SAS', 'OKC', 'UTA', 'SAC', 'POR',
          'DEN', 'PHX', 'DAL', 'ATL', 'MIA', 'LAC']
window = 4 # 4 samples per game (one sample / quarter)

In [3]:
N_players = 5
players = [f'player{i}' for i in range(1, N_players+1)]
n_components = 3 # num of hidden states
n_features = 3 # num of observed states
O_symbols = [0, 1, 2] # under-, avg-, over- performance
H_symbols = [0, 1, 2] # corresponding mental states
T = 100
learning_iterations = 100

In [4]:
# === M ===
avg_transO = np.array([[0.5, 0.3, 0.2],
                       [0.25, 0.5, 0.25],
                       [0.2, 0.3, 0.5]])

star_transO = np.array([[0.7, 0.3, 0],
                        [0.1, 0.8, 0.1],
                        [0, 0.3, 0.7]])
# === N ===
avg_transH = np.array([[0.8, 0.2, 0.0],
                       [0.1, 0.8, 0.1],
                       [0.0, 0.2, 0.8]])

# === emission_prob ===
avg_emission = np.array([[0.7, 0.3, 0],
                       [0.1, 0.8, 0.1],
                       [0.0, 0.3, 0.7]])

# ===  R ===
R_singleH = np.array([1] + [0] * len(players))
def R_singleHO(player):
    i = int(player[-1])
    arr = [0] * (len(players)+1)
    arr[0] = 0.7
    arr[i] = 0.3
    return np.array(arr)

def R_singleO(player):
    i = int(player[-1])
    arr = [0] * (len(players)+1)
    arr[i] = 1
    return np.array(arr)

def R_star(player, star):
    if player == star:
        return R_singleH
    arr = [0] * (len(players)+1)
    i = int(star[-1])
    arr[i] = 1
    return np.array(arr)
    
R_uniform = np.array([1/(1 + len(players))] * (len(players) + 1))

In [5]:
M = star_transO
N = avg_transH
E = avg_emission
R = {player: R_star(player, 'player1') for player in players}
initial_dist = np.array([0, 1, 0])

In [35]:
team = 'DAL'
observations = dict()

with open(f'../team-data/observations/{team}_observations.pickle', 'rb') as file:
    observations = pickle.load(file)

In [36]:
def cond(player, h1, h2, t, M_, N_, R_): # P(H_t^player = h1 | H_{t-1}^player = h2, O_{t-1})    
    # Requires M_, N_, R_, Os
    v = [N_[h2][h1]]
    for teammate in players:
        v.append(M_[Os[teammate][t-1]][h1])
            
    v = np.array(v)
    return np.dot(R_[player], v) 

In [37]:
def reconstruct_hidden(Os, M, N, E, R):    
    delta = {p: dict() for p in players}
    psi = {p: dict() for p in players}
    for p in players:
        delta[p][0] = np.array([0, 1, 0])
        psi[p][0] = np.array([None, None, None])

    for p in players:  
        for t in range(1, T):
            max_prev = np.array([np.max([delta[p][t-1][h_] * cond(p, h, h_, t, M, N, R) for h_ in H_symbols]) for h in H_symbols])
            delta[p][t] = [E[h][Os[p][t]] * max_prev[h] for h in H_symbols]
            psi[p][t] = np.array([np.argmax([delta[p][t-1][h_] * cond(p, h, h_, t, M, N, R) for h_ in H_symbols]) for h in H_symbols])

    Hs_predicted = {p: [] for p in players}
    for p in players:
        h_final = np.argmax([delta[p][T-1][h] for h in H_symbols])
        Hs_predicted[p] = [h_final]
        h = psi[p][T-1][h_final]
        for t in range(T-2, -1, -1):
            Hs_predicted[p] = [h] + Hs_predicted[p]
            h = psi[p][t][h]
            
    return Hs_predicted

In [38]:
with open(f'./profiles/{team}_profile.pkl', 'rb') as file:
        params = pickle.load(file)
print(params)
M, N, E, R = params['M'], params['N'], params['E'], params['R']
k = list(observations.keys())[0]
players = list(observations[k].keys())

Os = {p: [] for p in players}
for game_id in observations:
    game_stats = observations[game_id]
    for p in players:
        Os[p].extend(game_stats[p])

Hs_predicted = reconstruct_hidden(Os, M, N, E, R)

{'M': array([[0.6, 0.3, 0.1],
       [0.2, 0.6, 0.2],
       [0.1, 0.3, 0.6]]), 'N': array([[0.9, 0.1, 0. ],
       [0.1, 0.8, 0.1],
       [0. , 0.1, 0.9]]), 'E': array([[0.7, 0.2, 0.1],
       [0.1, 0.8, 0.1],
       [0.1, 0.2, 0.7]]), 'R': {'Luka Doncic': array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667]), 'Jalen Brunson': array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667]), 'Reggie Bullock': array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667]), 'Dorian Finney-Smith': array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667]), 'Dwight Powell': array([0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
       0.16666667])}, 'name': 'synthetic-338'}


In [40]:
def find_timestamps(timeseries):
    # Find timestamps of team collapse (ingore continuous 0, only consider the time of the first 0)
    indices = np.where(timeseries < 2)[0]
    if not indices.any():
        return []
    
    timestamps = [indices[0]]
    for i in range(1, len(indices)): # remove contiguous indices
        if indices[i] != indices[i-1] + 1:
            timestamps.append(indices[i])
    return timestamps

In [43]:
Hs_team_predicted = np.array([sum(states) for states in list(zip(*[Hs_predicted[p] for p in players]))])

In [44]:
find_timestamps(Hs_team_predicted)

[3, 82]

In [26]:
Os_DAL = Os_teams[team]

In [30]:
Os_DAL.keys()

dict_keys(['0022100014', '0022100029', '0022100052', '0022100069', '0022100075', '0022100089', '0022100104', '0022100116', '0022100136', '0022100150', '0022100163', '0022100181', '0022100204', '0022100222', '0022100237', '0022100248', '0022100266', '0022100298', '0022100310', '0022100325', '0022100338', '0022100345', '0022100363', '0022100373', '0022100383', '0022100400', '0022100410', '0022100420', '0022100455', '0022100468', '0022100481', '0022100492', '0022100506', '0022100524', '0022100531', '0022100550', '0022100558', '0022100572', '0022100586', '0022100603', '0022100622', '0022100639', '0022100650', '0022100663', '0022100675', '0022100682', '0022100705', '0022100720', '0022100732', '0022100746', '0022100755', '0022100778', '0022100791', '0022100807', '0022100820', '0022100835', '0022100853', '0022100868', '0022100886', '0022100903', '0022100917', '0022100931', '0022100943', '0022100954', '0022100972', '0022100988', '0022100999', '0022101014', '0022101036', '0022101046', '00221010

In [33]:
Os_DAL['0042100312']

{'Luka Doncic': [2, 0, 1, 1],
 'Jalen Brunson': [1, 2, 2, 1],
 'Reggie Bullock': [2, 2, 1, 1],
 'Dorian Finney-Smith': [2, 1, 1, 0],
 'Dwight Powell': [1, 1, 1, 0]}